In [3]:
import numpy as np, pandas as pd, joblib
import pennylane as qml
import pennylane.numpy as pnp
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# --- Global config ---
SEED = 7
EPOCHS = 300
BATCH_SIZE = 32
LR = 0.08
NOISE_P = 0.02
SHOTS = None   # set 4096 for realism
rng = np.random.default_rng(SEED)

# Choose number of qubits (should match PCA components)
N_QUBITS = 4
K_BLOCKS = 6   # number of variational layers

# --- Load MNIST dataset (replaces Iris/Wine) ---
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA

def load_dataset(name="mnist"):
    if name != "mnist":
        raise ValueError("Only 'mnist' is supported now")

    print("Loading MNIST...")
    mnist = fetch_openml("mnist_784", version=1, as_frame=False)
    X, y = mnist.data / 255.0, mnist.target.astype(int)

    # Reduce to N_QUBITS features for quantum input
    pca = PCA(n_components=N_QUBITS)
    X = pca.fit_transform(X)

    # Subsample for quicker training (optional)
    X, y = X[:5000], y[:5000]

    return X, y, [f"pca{i}" for i in range(N_QUBITS)], np.unique(y)

# Now load MNIST
X, y, feature_names, class_names = load_dataset("mnist")

X_tr, X_te, y_tr, y_te = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=SEED
)

print("Train shape:", X_tr.shape, "Test shape:", X_te.shape)



Loading MNIST...
Train shape: (4000, 4) Test shape: (1000, 4)


load dataset

QNN model (variable qubits)

In [12]:
dev = qml.device("default.mixed", wires=N_QUBITS, shots=SHOTS)

@qml.qnode(dev, interface="autograd")
def qnn_feature_map(x, theta):
    embed_block(x)        # your feature embedding
    var_block(theta)      # variational block
    return [qml.expval(qml.PauliZ(q)) for q in range(N_QUBITS)]


Training helpers

In [13]:
# Initialize random parameters for QNN
theta_init = rng.normal(size=(N_QUBITS, 3), scale=0.1)

def extract_features(X, theta):
    feats = []
    for x in X:
        feats.append(qnn_feature_map(x, theta))
    return np.array(feats)

print("Extracting QNN features...")
X_tr_feats = extract_features(X_tr[:2000], theta_init)   # subsample for speed
X_te_feats = extract_features(X_te[:1000], theta_init)

print("Feature shape:", X_tr_feats.shape)


Extracting QNN features...
Feature shape: (2000, 4)


Train OVR QNN

In [17]:
# Train a classical softmax classifier on QNN features
clf = LogisticRegression(max_iter=500, multi_class="multinomial", solver="lbfgs")
clf.fit(X_tr_feats, y_tr[:2000])

# Evaluate
yhat_tr = clf.predict(X_tr_feats)
yhat_te = clf.predict(X_te_feats)

print("\nHybrid QNN + Logistic Regression Results:")
print("Train acc:", accuracy_score(y_tr[:2000], yhat_tr))
print("Test  acc:", accuracy_score(y_te[:1000], yhat_te))
print("\nConfusion matrix (test):\n", confusion_matrix(y_te[:1000], yhat_te))
print("\nReport (test):\n", classification_report(y_te[:1000], yhat_te, target_names=[str(c) for c in class_names]))


# Save QNN parameters + classical classifier
hybrid_model = {"theta": theta_init, "classifier": clf}
joblib.dump(hybrid_model, "hybrid_qnn_mnist.joblib")
print("Hybrid QNN model saved as hybrid_qnn_mnist.joblib")





Hybrid QNN + Logistic Regression Results:
Train acc: 0.3195
Test  acc: 0.288

Confusion matrix (test):
 [[37  1 10  1  7  1  4 19 16  0]
 [ 1 83 11  0  6  0  8  1  2  0]
 [ 9  4 25  0  0  1 39 12  5  3]
 [23 26  6  0  6  2 11 11 14  0]
 [29 10  4  0 11  1  5 28 19  0]
 [15  6  5  0  8  1 11 12 29  0]
 [ 2 11 19  2  1  0 50  5 10  0]
 [21  1 15  2  5  0  6 50  9  1]
 [17 15  2  0 10  0  1 17 30  0]
 [23  6  5  0  5  0 10 37 12  1]]

Report (test):
               precision    recall  f1-score   support

           0       0.21      0.39      0.27        96
           1       0.51      0.74      0.60       112
           2       0.25      0.26      0.25        98
           3       0.00      0.00      0.00        99
           4       0.19      0.10      0.13       107
           5       0.17      0.01      0.02        87
           6       0.34      0.50      0.41       100
           7       0.26      0.45      0.33       110
           8       0.21      0.33      0.25        92
      

c:\Users\shoai\qml-env\Lib\site-packages\sklearn\linear_model\_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt

# --- Noise block with multiple noise types ---
def noise_block(p, noise_type="depolarizing"):
    if p <= 0:
        return
    for q in range(N_QUBITS):
        if noise_type == "depolarizing":
            qml.DepolarizingChannel(p, wires=q)
        elif noise_type == "bitflip":
            qml.BitFlip(p, wires=q)
        elif noise_type == "phaseflip":
            qml.PhaseFlip(p, wires=q)
        elif noise_type == "amplitude":
            qml.AmplitudeDamping(p, wires=q)
        elif noise_type == "phase":
            qml.PhaseDamping(p, wires=q)

# --- QNN with noise ---
@qml.qnode(dev, interface="autograd")
def qnn_with_noise(x, theta, p=0.0, noise_type="depolarizing"):
    embed_block(x)
    var_block(theta)
    noise_block(p, noise_type)
    return [qml.expval(qml.PauliZ(q)) for q in range(N_QUBITS)]

# --- Feature extraction under noise ---
def extract_features_with_noise(X, theta, p=0.0, noise_type="depolarizing"):
    feats = []
    for x in X:
        feats.append(qnn_with_noise(x, theta, p=p, noise_type=noise_type))
    return np.array(feats)

# --- Robustness experiment ---
noise_types = ["depolarizing", "bitflip", "phaseflip", "amplitude", "phase"]
p_values = np.linspace(0, 1, 11)  # [0.0, 0.1, ..., 1.0]
results = {nt: [] for nt in noise_types}

# Load your hybrid model (theta + classifier)
hybrid_model = joblib.load("hybrid_qnn_mnist.joblib")
theta = hybrid_model["theta"]
clf = hybrid_model["classifier"]

print("\nRunning robustness experiments on Hybrid QNN...")

for nt in noise_types:
    for p in p_values:
        X_te_feats_noisy = extract_features_with_noise(X_te[:500], theta, p=p, noise_type=nt)
        y_pred = clf.predict(X_te_feats_noisy)



Running robustness experiments...


NameError: name 'theta' is not defined